# IBM Data Science Capstone Project
-----------------------------------

## *Introduction Section & Problem Statement*

Analysis of the Bar Location & Nightlife Sectors in Singapore for Re-Opening in Next Phase of Covid-19 

Describe the data that you will be using to solve the problem or execute your idea. Remember that you will need to use the Foursquare location data to solve the problem or execute your idea. You can absolutely use other datasets in combination with the Foursquare location data. So make sure that you provide adequate explanation and discussion, with examples, of the data that you will be using, even if it is only Foursquare location data. All these questions and more will be answered below. The whole process will be carried out via the below plan:

1) Identify the Names of the Bars in Singapore and its Locations through Web-Scrapping.

2) Use Foresquare Data to Obtain info about Most Popular Venues.

3) Use Clustering to identify Close-Proximity of Bar Locations.

4) Visualization using Chloropleth Map.